In [ ]:
## requirements

In [ ]:
# !pip install azure
# !pip install azure-kusto-data
# !pip install azure-kusto-ingest


In [8]:
import pandas as pd
import numpy as np
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.ingest import  QueuedIngestClient,IngestionProperties
from azure.kusto.data.helpers import dataframe_from_result_table

In [ ]:
cluster= "https://<ClusterName>.<Region>.kusto.windows.net/"
ingest_uri="https://ingest-<ClusterName>.<Region>.kusto.windows.net/"
# aap_id=""
database_name="<database_name>"
app_id="app_id"
client_secret="secret"
tenant_id="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxx"
subscription_id="xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

In [ ]:
kcsb_ingest=KustoConnectionStringBuilder.with_aad_application_key_authentication(ingest_uri, app_id, client_secret, tenant_id)
kcsb_data = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, app_id, client_secret, tenant_id)
kusto_client = KustoClient(kcsb_data)
kusto_client_ingest=KustoClient(kcsb_ingest)
create_table_cmd = ".create table testEvents (StartTime: datetime, EndTime: datetime, Id: int, State: string)"


In [ ]:
## Data projection 

In [ ]:

select_query="<table_name> | sort by serial_id desc | take 10"
# execute command
select_response =kusto_client.execute(database_name, select_query)

df_app_select = dataframe_from_result_table(select_response.primary_results[0])
df_app_select

In [ ]:
## Insert command

In [ ]:
df_sample_table= df_app_select[["date_end","date_start","id"]]

# kusto client insert
kcsb_insert = KustoConnectionStringBuilder.with_aad_application_key_authentication(connection_string=ingest_uri, aad_app_id=app_id, app_key=client_secret, authority_id=tenant_id)
adx_client = QueuedIngestClient(kcsb_insert)
ingestion_props = IngestionProperties(database=database_name,table=['sample'])

adx_client.ingest_from_dataframe(df_sample_table, ingestion_properties=ingestion_props)